In [1]:
def evaluate(df, median_item_degree, item_degree_map, num_cases_full, num_cases_half):
    gg = df.groupby(['user_id'])

    ndcg_50_full = 0.0
    hitrate_50_full = 0.0

    ndcg_50_half = 0.0
    hitrate_50_half = 0.0

    for _, g in tqdm(gg):
        try:
            item_id = g[g['label'] == 1]['item_id'].values[0]
            item_degree = item_degree_map[item_id]
        except Exception as e:
            continue

        predictions = g['item_id'].values.tolist()

        rank = 0
        while rank < 50 and predictions[rank] != item_id:
            rank += 1

        if rank < 50:
            ndcg_50_full += 1.0 / np.log2(rank + 2.0)
            hitrate_50_full += 1.0

        if item_degree <= median_item_degree:
            if rank < 50:
                ndcg_50_half += 1.0 / np.log2(rank + 2.0)
                hitrate_50_half += 1.0

    ndcg_50_full /= num_cases_full
    hitrate_50_full /= num_cases_full

    ndcg_50_half /= num_cases_half
    hitrate_50_half /= num_cases_half

    return ndcg_50_full, hitrate_50_full, ndcg_50_half, hitrate_50_half

In [1]:
def evaluate_scores(df, phase):
    df_qtime = pd.read_pickle('../user_data/data/qtime.pkl')
    df_click = pd.read_pickle('../user_data/data/click.pkl')
    
    df_qtime = df_qtime[df_qtime['phase']==phase]
    df_click = df_click[df_click['phase']==phase]

    oof_answer = df_qtime[df_qtime['item_id'] != -1]
    num_cases_full = oof_answer.shape[0]

    answer_items = oof_answer['item_id'].values
    item_degree = df_click['item_id'].value_counts().reset_index()
    item_degree.columns = ['item_id', 'degree']

    answer_item_degree = item_degree[item_degree['item_id'].isin(answer_items)]
    list_item_degress = answer_item_degree['degree'].values.tolist()
    median_item_degree = list_item_degress[len(list_item_degress) // 2]
    item_degree_map = dict(zip(item_degree['item_id'], item_degree['degree']))
    num_cases_half = answer_item_degree[answer_item_degree['degree']
                                        <= median_item_degree].shape[0]

    ndcg_50_full, hitrate_50_full, ndcg_50_half, hitrate_50_half = evaluate(
        df, median_item_degree, item_degree_map, num_cases_full, num_cases_half)
    return ndcg_50_full, hitrate_50_full, ndcg_50_half, hitrate_50_half